In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import nltk
import string
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity
import networkx as nx
nltk.download('punkt')
nltk.download('stopwords')
from farasa.stemmer import FarasaStemmer
import langid

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
pip  install -U farasapy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [95]:
df = pd.read_csv(r"C:\Users\Mohammad El Hakawati\Downloads\Complaints dataset.csv")


C:\Users\Mohammad El Hakawati\AppData\Local\Temp\ipykernel_4048\806539116.py:1: DtypeWarning: Columns (15,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Mohammad El Hakawati\Downloads\Complaints dataset.csv")


In [96]:
df = df[df['CASE_DESC'].str.len() >= 5]

In [102]:
pip install farasa


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
i_path=r"C:\Users\Mohammad El Hakawati\Downloads\technical-support.png"

In [3]:
import tkinter as tk
from tkinter import messagebox

In [4]:
def calculate_distance():
    selected_option = search_option.get()

    # Assuming lowest_euclidean, lowest_manhattan, and highest_cosine are defined somewhere in your code

    new_window = tk.Toplevel()  # Create a new window

    if selected_option == 1:  # Euclidean Distance (basic string similarity)
        result_label = tk.Label(new_window, text="lowest_euclidean[['CASE_DESC', 'Euclidean Distances']]")
        result_label.pack()
    elif selected_option == 2:  # Manhattan Distance (word count difference)
        result_label = tk.Label(new_window, text="lowest_manhattan[['CASE_DESC', 'Manhattan Distances']]")
        result_label.pack()
    elif selected_option == 3:  # Cosine Similarity (basic overlap of keywords)
        result_label = tk.Label(new_window, text="highest_cosine[['CASE_DESC', 'Cosine Similarity']]")
        result_label.pack()
    else:
        result_label = tk.Label(new_window, text="Invalid measure selected.")
        result_label.pack()


In [5]:

root = tk.Tk()
root.title("Student Info System")
i_path = r"C:\Users\Mohammad El Hakawati\Downloads\technical-support.png"
img = tk.PhotoImage(file=i_path)
img = img.subsample(1, 1)

colored_box = tk.Canvas(root, width=root.winfo_screenwidth(), height=30, bg='#81b0d4')
colored_box.pack(side="top", fill=tk.X)

# Create a label inside the canvas with a larger font size
label_id = tk.Label(colored_box, text="Enter Student ID", font=('Helvetica', 25), fg='#023047', bg='#81b0d4')
label_id.pack(pady=5)

label_img = tk.Label(root, image=img)
label_img.pack(side="right", padx=10, pady=10)

label_id = tk.Label(root, text="Enter your query below", font=('Helvetica', 20), fg='#023047', bg='#81b0d4')
label_id.pack(side="top", pady=50)

entry_id = tk.Entry(root)
entry_id.pack(side="top", ipadx=170, ipady=10)

search_option = tk.IntVar()

# Create a frame to hold the radio buttons
radio_frame = tk.Frame(root)

radio_euclidean = tk.Radiobutton(radio_frame, text="Euclidean Distance", font=('Helvetica', 17), fg='#023047', variable=search_option, value=1)
radio_manhattan = tk.Radiobutton(radio_frame, text="Manhattan Distance", font=('Helvetica', 17), fg='#023047', variable=search_option, value=2)
radio_cosine = tk.Radiobutton(radio_frame, text="Cosine Similarity", font=('Helvetica', 17), fg='#023047', variable=search_option, value=3)

radio_euclidean.pack(side="top", padx=400, pady=5, ipadx=50)
radio_manhattan.pack(side="top", padx=400, pady=5, ipadx=50)
radio_cosine.pack(side="top", padx=400, pady=5, ipadx=50)

# Create the button with the command set to calculate_distance
b = tk.Button(radio_frame, text="Search", command=calculate_distance, font=('Helvetica', 17), fg='#023047', bg='#81b0d4')
b.pack(side="bottom", pady=10)

# Pack the radio_frame into the root window
radio_frame.pack(side="left", padx=20, pady=20)
root.mainloop()


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

def preprocess_text(text):
    def tokenize_arabic_text(text):
        # Tokenization: Detect and isolate individual words
        words = re.findall(r'\b\w+\b', text)
        return words

    def lemmatize_single_word(word):
        lemmatizer = WordNetLemmatizer()
        lemmatized_word = lemmatizer.lemmatize(word)
        return lemmatized_word

    def word_representation(word):
        # Word representation: Introduce "EXTRA-SEKOUN" diacritical mark for missed diacritics
        # Assume the input word is a sequence of Arabic letters and diacritical marks
        new_word = ''
        for char in word:
            if char.isalpha() or char in ['.', 'EXTRA-SEKOUN']:
                new_word += char
        return new_word

    def remove_definite_article(word):
        # Regular expression pattern to match Arabic words
        arabic_word_pattern = re.compile(r'\b\w+\b', re.U)

        # Check if the word is in Arabic and contains the definite article 'ال'
        if re.match(arabic_word_pattern, word) and word.startswith('ال'):
            # Remove the definite article
            word_without_article = word[len('ال',):].replace('و', '')
            return word_without_article
        else:
            # Return the original word if it doesn't start with 'ال' or if it's not an Arabic word
            return word

    def reduce_orthographic_ambiguity(word):
        # Define mapping for orthographic ambiguity reduction
        orthographic_mapping = {
            'ة': 'ه',   # Taa marbuta to Ha
            'ؤ': 'ء',   # Waw hamza to Hamza
            'إ': 'ا',   # Alef hamza below to Alef
            'أ': 'ا',   # Alef hamza above to Alef
            'ء': 'ا',   # Hamza to Alef
            'ئ': 'ء',   # Yeh hamza to Hamza
            'ٱ': 'ا',   # Alef wasla to Alef
            'آ': 'ا'    # Alef with madda to Alef
        }

        # Replace characters based on the defined mapping
        for original, replacement in orthographic_mapping.items():
            word = word.replace(original, replacement)

        return word

    def keep_specific_numeric_patterns(text):
        # Your implementation to keep specific numeric patterns
        # For example, you can use regular expressions to remove certain numeric patterns
        # Replace the following line with your implementation
        processed_text = re.sub(r'\b\d+\b', '', text)
        return processed_text

    # Tokenization
    tokens = nltk.word_tokenize(text)
    # 3. Remove punctuation and short words, excluding consecutive dots
    tokens = [token for token in tokens if re.match(r'\w', token) and len(token) > 1 and not re.match(r'\.{2,}', token)]

    # Remove punctuation for both English and Arabic text
    english_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
    arabic_tokens = [token for token in tokens if re.search('[\u0600-\u06FF]', token)]

    # Preprocess Arabic and English tokens
    processed_tokens = []
    for token in tokens:
        if re.match(r'^(079|078|077|00962|962|\+962)\d+$', token):
            processed_tokens.append(token)
        elif re.search('[\u0600-\u06FF]', token):  # Check if the token contains Arabic characters
            # Apply Arabic preprocessing steps
            arabic_tokens = tokenize_arabic_text(token)
            arabic_tokens = [reduce_orthographic_ambiguity(word_representation(remove_definite_article(word))) for word in arabic_tokens]
            processed_tokens.extend(arabic_tokens)
        else:
            # Apply English preprocessing steps
            token = lemmatize_single_word(token.lower())
            processed_tokens.append(token)

    # Remove stopwords
    stop_words_arabic = set(stopwords.words('arabic'))
    most_popular_stopwords_arabic = {"هي", "هي","على", "من", "لم", "تم", ".", "،", "أنا", "أنت", "إلى", "في", "عن", "هذا", "هذه", "هو", "وهي", "ثم", ".."}
    combined_stops_arabic = stop_words_arabic.union(most_popular_stopwords_arabic)
    processed_tokens = [token for token in processed_tokens if token not in combined_stops_arabic]

    stop_words_english = set(stopwords.words('english'))
    processed_tokens = [token for token in processed_tokens if token not in stop_words_english]

    # Apply stemming
    stemmer = PorterStemmer()
    processed_tokens = [stemmer.stem(token) for token in processed_tokens]

    # Join tokens back into text
    processed_text = ' '.join(processed_tokens)

    # Apply the specific numeric pattern removal logic
    processed_text = keep_specific_numeric_patterns(processed_text)

    return processed_text

# Assuming 'df' is a DataFrame with a column named 'CASE_DESC'
# Apply the preprocess_text function to the 'CASE_DESC' column
df['CASE_DESC'] = df['CASE_DESC'].apply(preprocess_text)

# Print the resulting DataFrame
print(df['CASE_DESC'])